# Práctica 5 Visión por Computador
## Autores:
- Héctor Wood Santana
- Alejandro Viera Ruiz 

# Tarea.
Tras mostrar opciones para la detección y extracción de información de caras humanas con deepface, la tarea a entregar consiste en proponer un escenario de aplicación y desarrollar un prototipo de temática libreque provoque reacciones a partir de la información extraida del rostro. Los detectores proporcionan información del rostro, y de sus elementos faciales. Ideas inmediatas pueden ser filtros, aunque no hay limitaciones en este sentido. La entrega debe venir acompañada de un gif animado o vídeo de un máximo de 30 segundos con momentos seleccionados de la propuesta. Se utilizará para una posterior votación y elección de las mejores entre el grupo.

In [3]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import numpy as np

In [4]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

# Face detection and eye model setup
imodoF = 2  # DNN 
imodoE = 1  # DLIB68

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

# Modos
mode = 0

# Cargamos las imagenes de las gafas
gafas = cv2.imread('gafas1sinfondo.png', cv2.IMREAD_UNCHANGED)

# Imprimir información sobre la imagen cargada
if gafas is not None:
    print("Dimensiones de la imagen de gafas:", gafas.shape)
else:
    print("Error: La imagen de las gafas no se pudo cargar. Verifica la ruta o el archivo.")

# Verifica si las gafas tienen canal alfa; si no, añade uno manualmente
if gafas.shape[2] == 3:  # Si no tiene canal alfa
    gafas = cv2.cvtColor(gafas, cv2.COLOR_BGR2BGRA)


# Cargamos las imagenes del sombrero
sombrero = cv2.imread('sombrerosinfondo.png', cv2.IMREAD_UNCHANGED)

# Imprimir información sobre la imagen cargada
if sombrero is not None:
    print("Dimensiones de la imagen del sombrero:", sombrero.shape)
else:
    print("Error: La imagen del sombrero no se pudo cargar. Verifica la ruta o el archivo.")

# Verifica si las gafas tienen canal alfa; si no, añade uno manualmente
if sombrero.shape[2] == 3:  # Si no tiene canal alfa
    sombrero = cv2.cvtColor(sombrero, cv2.COLOR_BGR2BGRA)

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()

    if ret:
        # For HS normalization
        B, G, R = cv2.split(frame)

        # Search face with a specific setup for face and eye detection
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
        if values is not None:
            face, eyes, shape = values

            #draws face container
            [x, y , w, h] = face
            if x > -1:
                

                # draws eyes and mask if available
                [lex, ley, rex, rey] = eyes
                if lex > -1 and rex > -1:
                    
                    
                    

                    #Cambiamos de modo
                    if mode == 0:
                        # Show detected facial elements (Dibuja puntos de contorno de la cara)
                        # Bounding Box de la cara
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                        if imodoF > 0:
                            for (x, y) in shape:
                                cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                        #Puntos del centro de los ojos
                        cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                        cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                    elif mode == 1:
                        # Calculamos la distancia entre los ojos
                        distancia_ojos = int(np.sqrt((rex - lex) ** 2 + (rey - ley) ** 2))
                        # Redimensionamos las gafas
                        gafas_resized = cv2.resize(gafas, (int(distancia_ojos * 2.5), int(distancia_ojos * 1.2)))                    
                        # Calculo del las posiciones de la gafa
                        y_offset = int((ley + rey) / 2 - gafas_resized.shape[0] / 2)
                        x_offset = int((lex + rex) / 2 - gafas_resized.shape[1] / 2)
                        
                        # Cálculo de transparencia y canales RGB
                        alpha = gafas_resized[:, :, 3] / 255.0  # Canal alfa (normalizado entre 0 y 1)
                        gafas_rgb = gafas_resized[:, :, :3]  # Los colores RGB (sin el canal alfa)

                        # Calcular los índices de la región del frame donde se va a superponer la imagen
                        y_start = max(y_offset, 0)
                        y_end = min(y_offset + gafas_resized.shape[0], frame.shape[0])
                        x_start = max(x_offset, 0)
                        x_end = min(x_offset + gafas_resized.shape[1], frame.shape[1])

                        # Extraer el fragmento del frame que será modificado
                        frame_region = frame[y_start:y_end, x_start:x_end]

                        # Calcular el área que debe ser modificada (donde el alfa es mayor que 0)
                        alpha_mask = alpha[y_start - y_offset:y_end - y_offset, x_start - x_offset:x_end - x_offset]
                        frame[y_start:y_end, x_start:x_end] = (1 - alpha_mask[..., None]) * frame_region + alpha_mask[..., None] * gafas_rgb[y_start - y_offset:y_end - y_offset, x_start - x_offset:x_end - x_offset]

                    
                    
                    elif mode == 2:
                        # Redimensionar el sombrero en función del ancho del bounding box de la cara
                        ancho_sombrero = int(w * 2)
                        alto_sombrero = int(ancho_sombrero * sombrero.shape[0] / sombrero.shape[1]) 
                        sombrero_resized = cv2.resize(sombrero, (ancho_sombrero, alto_sombrero))
                        
                        # Calcular la posición para colocar el sombrero encima de la cara
                        x_offset = int(x + w / 2) - int(ancho_sombrero / 2)
                        y_offset = int(((y + (h/2)) - alto_sombrero))
                        
                        # Cálculo de transparencia y canales RGB
                        alpha = sombrero_resized[:, :, 3] / 255.0  # Canal alfa (normalizado entre 0 y 1)
                        sombrero_rgb = sombrero_resized[:, :, :3]  # Los colores RGB (sin el canal alfa)

                        # Calcular los índices de la región del frame donde se va a superponer la imagen
                        y_start = max(y_offset, 0)
                        y_end = min(y_offset + sombrero_resized.shape[0], frame.shape[0])
                        x_start = max(x_offset, 0)
                        x_end = min(x_offset + sombrero_resized.shape[1], frame.shape[1])

                        # Extraer el fragmento del frame que será modificado
                        frame_region = frame[y_start:y_end, x_start:x_end]

                        # Calcular el área que debe ser modificada (donde el alfa es mayor que 0)
                        alpha_mask = alpha[y_start - y_offset:y_end - y_offset, x_start - x_offset:x_end - x_offset]
                        frame[y_start:y_end, x_start:x_end] = (1 - alpha_mask[..., None]) * frame_region + alpha_mask[..., None] * sombrero_rgb[y_start - y_offset:y_end - y_offset, x_start - x_offset:x_end - x_offset]
                    
                    elif mode == 3:
                        # Calculamos la distancia entre los ojos
                        distancia_ojos = int(np.sqrt((rex - lex) ** 2 + (rey - ley) ** 2))
                        # Redimensionamos las gafas
                        gafas_resized = cv2.resize(gafas, (int(distancia_ojos * 2.5), int(distancia_ojos * 1.2)))
                        # Redimensionar el sombrero en función del ancho del bounding box de la cara
                        ancho_sombrero = int(w * 2)
                        alto_sombrero = int(ancho_sombrero * sombrero.shape[0] / sombrero.shape[1]) 
                        sombrero_resized = cv2.resize(sombrero, (ancho_sombrero, alto_sombrero))                    
                        # Calculo del las posiciones de la gafa
                        y_offset_gafa = int((ley + rey) / 2 - gafas_resized.shape[0] / 2)
                        x_offset_gafa = int((lex + rex) / 2 - gafas_resized.shape[1] / 2)
                        # Calcular la posición para colocar el sombrero encima de la cara
                        x_offset_sombrero = int(x + w / 2) - int(ancho_sombrero / 2)
                        y_offset_sombrero = int(((y + (h/2)) - alto_sombrero))
                        # Superposición de las gafas
                        # Cálculo de transparencia y canales RGB
                        alpha = gafas_resized[:, :, 3] / 255.0  # Canal alfa (normalizado entre 0 y 1)
                        gafas_rgb = gafas_resized[:, :, :3]  # Los colores RGB (sin el canal alfa)

                        # Calcular los índices de la región del frame donde se va a superponer la imagen
                        y_start = max(y_offset_gafa, 0)
                        y_end = min(y_offset_gafa + gafas_resized.shape[0], frame.shape[0])
                        x_start = max(x_offset_gafa, 0)
                        x_end = min(x_offset_gafa + gafas_resized.shape[1], frame.shape[1])

                        # Extraer el fragmento del frame que será modificado
                        frame_region = frame[y_start:y_end, x_start:x_end]

                        # Calcular el área que debe ser modificada (donde el alfa es mayor que 0)
                        alpha_mask = alpha[y_start - y_offset_gafa:y_end - y_offset_gafa, x_start - x_offset_gafa:x_end - x_offset_gafa]
                        frame[y_start:y_end, x_start:x_end] = (1 - alpha_mask[..., None]) * frame_region + alpha_mask[..., None] * gafas_rgb[y_start - y_offset_gafa:y_end - y_offset_gafa, x_start - x_offset_gafa:x_end - x_offset_gafa]
                        # Superponer el sombrero en el frame usando transparencia (canal alfa)
                        # Cálculo de transparencia y canales RGB
                        alpha = sombrero_resized[:, :, 3] / 255.0  # Canal alfa (normalizado entre 0 y 1)
                        sombrero_rgb = sombrero_resized[:, :, :3]  # Los colores RGB (sin el canal alfa)

                        # Calcular los índices de la región del frame donde se va a superponer la imagen
                        y_start = max(y_offset_sombrero, 0)
                        y_end = min(y_offset_sombrero + sombrero_resized.shape[0], frame.shape[0])
                        x_start = max(x_offset_sombrero, 0)
                        x_end = min(x_offset_sombrero + sombrero_resized.shape[1], frame.shape[1])

                        # Extraer el fragmento del frame que será modificado
                        frame_region = frame[y_start:y_end, x_start:x_end]

                        # Calcular el área que debe ser modificada (donde el alfa es mayor que 0)
                        alpha_mask = alpha[y_start - y_offset_sombrero:y_end - y_offset_sombrero, x_start - x_offset_sombrero:x_end - x_offset_sombrero]
                        frame[y_start:y_end, x_start:x_end] = (1 - alpha_mask[..., None]) * frame_region + alpha_mask[..., None] * sombrero_rgb[y_start - y_offset_sombrero:y_end - y_offset_sombrero, x_start - x_offset_sombrero:x_end - x_offset_sombrero]


                    # Normalize and show
                    # color channels
                    normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                    Bnorm = normalizatorHS.normf_image
                    normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                    Gnorm = normalizatorHS.normf_image
                    normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                    Rnorm = normalizatorHS.normf_image
                    NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                    cv2.imshow("Normalized", NormBGR)

        # Show resulting image
        cv2.imshow('Cam', frame)
        
        # Esc to finish
        tec = cv2.waitKey(40)
        if tec & tec == 27:  # Esc
            break
        elif tec & 0xFF == ord('0'):
            mode = 0
        elif tec & 0xFF == ord('1'):
            mode = 1
        elif tec & 0xFF == ord('2'):
            mode = 2
        elif tec & 0xFF == ord('3'):
            mode = 3
            

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
Dimensiones de la imagen de gafas: (433, 577, 4)
Dimensiones de la imagen del sombrero: (500, 500, 4)
